In [2]:
import csv
from scapy.all import rdpcap

In [8]:
def extract_tcp_data(pcap_file, csv_file, target_ip):
    # Read the packets from the pcap file
    packets = rdpcap(pcap_file)
    
    # Open the CSV file for writing
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['Timestamp', 'Data Segment'])

        # Iterate through the packets and extract the TCP payload and timestamp
        for packet in packets:
            if packet.haslayer("IP") and packet.haslayer("TCP"):
                if packet["IP"].dst == target_ip:
                    tcp_payload = bytes(packet["TCP"].payload).hex()
                    if len(tcp_payload) > 4:
                        timestamp = packet.time
                        # Write the timestamp and data segment to the CSV file
                        writer.writerow([timestamp, tcp_payload])

# Replace 'example.pcap' with the path to your pcap file
# Replace 'output.csv' with the desired output CSV file name
# Replace '192.168.228.5' with the target IP address

In [7]:
pcap_file = "./ezsocket_all_apipcap_dataset2.pcap"
csv_file = "./ezsocket_all_apipcap_dataset2_time_data.csv"
target_ip = "192.168.228.70"
extract_tcp_data(pcap_file, csv_file, target_ip)

In [41]:
import pandas as pd
import linecache

def get_line_context(file_path, line_number):
    return linecache.getline(file_path, line_number).strip()

def match_timestamps_sequentially(df1, df2, tolerance=0.5):
    matched_data = []
    j = 0
    len_df2 = len(df2)

    for i in range(len(df1)):
        timestamp1 = df1.loc[i, 'Timestamp']

        while j < len_df2 and df2.loc[j, 'Timestamp'] < timestamp1:
            j += 1
        
        k = j
        while k < len_df2 and df2.loc[k, 'Timestamp'] <= timestamp1 + tolerance:
            matched_data.append({
                
                'Function and Parameters': get_line_context('./ezsocket_api_parameter_timestamp_dataset2.csv', i + 2),
                #'Function and Parameters': df1.loc[i],
                'Data Segment': df2.loc[k, 'Data Segment']
            })
            k += 1

    return pd.DataFrame(matched_data)

In [42]:
# Read the first CSV file
df1 = pd.read_csv('./ezsocket_api_parameter_timestamp_dataset2.csv')

# Read the second CSV file
df2 = pd.read_csv('./ezsocket_all_apipcap_dataset2_time_data.csv')

# Convert Timestamp columns to float for comparison
df1['Timestamp'] = df1['Timestamp'].astype(float) / 1000
df2['Timestamp'] = df2['Timestamp'].astype(float)


# Ensure df2 is sorted by Timestamp
#df2 = df2.sort_values(by='Timestamp').reset_index(drop=True)

# Match timestamps within a 1-second tolerance where timestamp2 > timestamp1
matched_df = match_timestamps_sequentially(df1, df2, tolerance=0.5)

# Save the matched dataframe to a new CSV file
matched_df.to_csv('merged_api_para_pcapdata_dataset2.csv', index=False)